In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import pywhatkit
import json

In [ ]:
model1 = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
cam = cv2.VideoCapture(0)

In [ ]:
count=100
while True:
    ret,photo = cam.read()
    faces = model1.detectMultiScale(photo)
    if len(faces) == 0:
        pass
    else:
        photo_cropped = cropper(photo)
        cv2.resize(photo_cropped,(200,200))
        cv2.imwrite("./faces/harsh/" +str(count)+".jpg",photo_cropped)
        count+=1
        
        if cv2.waitKey(1) == 13 or count == 200:
            break
cv2.destroyAllWindows()      

In [ ]:
def cropper(img):
    for (x1,y1,x2,y2) in faces:
        cropped = img[y1:y1+y2,x1:x1+x2]
        return cropped
    

In [ ]:
cam.release()

In [ ]:
training_data, labels = ([],[])

path = "./faces/harsh/"
location = [f for f in listdir(path) if isfile(join(path,f))]
for i,files in enumerate(location):
    image_path = path + location[i]
    images = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE )
    training_data.append(np.asarray(images, dtype=np.uint8))
    labels.append(i)

labels = np.asarray(labels,dtype=np.int32)

In [ ]:
model = cv2.face_LBPHFaceRecognizer.create()

In [ ]:
model.train(np.asarray(training_data),np.asarray(labels))

In [ ]:
cam = cv2.VideoCapture(0)
while True:
    ret,photo = cam.read()
    photo = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

    faces = model1.detectMultiScale(photo)
    if len(faces) == 0:
        pass
    else:
        x1,y1,x2,y2 = faces[0]
        square = cv2.rectangle(photo,(x1,y1),(x1+x2,y1+y2),[0,0,255],1)
        cv2.imshow('Camera', square)
        predict_photo = cropper(photo)
        x = model.predict(predict_photo)
        con = int(100 * (1 - (x[1])/400))

        if con>90:
            if x[0]<100:
                cv2.putText(square, str(con), (100,120), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,0))
                    pywhatkit.sendwhatmsg_instantly(phone_number,message_string)
            elif x[0]>100 and x[0]<200:
                cv2.putText(square, str(con), (100,120), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255))
                ec2  = os.popen("aws ec2 run-instances --image-id ami-0e5d82cae7458738b --instance-type t2.micro --count 1 --subnet-id subnet-1311e478 --security-group-ids sg-029e99a703a4fa089 --key-name AWS_key").read()
                ec2 = json.loads(ec2)
                ec2_id = ec2['Instances'][0]['InstanceId']
                gp = os.popen("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1a").read()
                gp = json.loads(gp)
                gp = gp['VolumeId']
                final = "aws ec2 attach-volume --volume-id "+ gp +" --instance-id "+ ec2_id +" --device /dev/sdf"
                os.system(final)
            

        if cv2.waitKey(1) == 13:
            break   
        
cv2.destroyAllWindows()
cam.release()